In [1]:
import pandas as pd
# Load MetaMap
from pymetamap import MetaMap

# Import os to make system calls
import os,re

# For pausing
from time import sleep
from pprint import pprint

In [2]:
path=os.getcwd() # for current path

In [3]:
def metamap(label):
    os.system(metamap_base_dir + metamap_pos_server_dir +" "+label) # Part of speech tagger
    os.system(metamap_base_dir + metamap_wsd_server_dir +" "+label) # Word sense disambiguation 



In [4]:


# Setup UMLS Server
metamap_base_dir = path+'/public_mm/'
metamap_bin_dir = 'bin/metamap'
metamap_pos_server_dir = 'bin/skrmedpostctl'
metamap_wsd_server_dir = 'bin/wsdserverctl'

# Start servers
metamap('start')
# Sleep a bit to give time for these servers to start up
sleep(60)

$Starting skrmedpostctl: 
started.
$Starting wsdserverctl: 
started.
loading properties file /home/sadika/public_mm/WSD_Server/config/disambServer.cfg
WSD Server initializing disambiguation methods.
WSD Server databases and disambiguation methods have been initialized.


In [5]:
sentence = "BRCA1 gene causes predisposition to breast cancer and ovarian cancer"


In [6]:
# Concept identification
def concept_identification(sentence):
   os.system("echo "+sentence+" | "+metamap_base_dir+"/"+metamap_bin_dir+" -I >"+path+"/out.txt")

   
def parse_metamap(filename):
 
        sema_type=[]
        cptnm=[]
        uid=[]   
        with open(filename,encoding='utf-8') as fl:
            for lines in (fl):
                regex=re.findall("^\s+\d{3,4}",lines)
                if bool(regex)==True:
                   
                    concept_name=re.findall(":[a-zA-Z0-9\s]*",lines)
                    cptnm.append(concept_name[0].lstrip(":").lower())
                  
                    cid=re.findall("C\d+",lines.strip()) 
                    if cid[0] not in uid:
                        uid.append(cid[0]) 
                        semantic_type=re.findall("\[(.*?)\]",lines)
                        if len(semantic_type)>1:
                            for item in semantic_type:
                                if item in umls_semantic:
                                    sema_type.append(item)
                            
                        else:
                            sema_type.append(semantic_type[0]) 

                    
    
      

              
        return cptnm,sema_type
   


In [7]:
concept_identification(sentence)
cptnm,cpt_sem=parse_metamap(path+"/out.txt")
print(cptnm,cpt_sem)

Berkeley DB databases (USAbase 2020AA strict model) are open.
Static variants will come from table varsan in /home/sadika/public_mm/DB/DB.USAbase.2020AA.strict.
Derivational Variants: Adj/noun ONLY.
Variant generation mode: static.
Established connection $stream(139848377257280) to TAGGER Server on localhost.

metamap (2020)

Control options:
  composite_phrases=4
  lexicon=db
  mm_data_year=2020AA
  show_cuis


['brca 1 gene ', 'causes ', 'causes ', 'cause ', 'breast cancer predisposition ', 'ovarian cancer ', 'ovarian cancer ', 'ovarian cancer '] ['Gene or Genome', 'Functional Concept', 'Functional Concept', 'Conceptual Entity', 'Finding', 'Neoplastic Process', 'Neoplastic Process', 'Neoplastic Process']


In [8]:
# Stop servers

metamap('stop')

$Stopping skrmedpostctl: 
Stopping Tagger Server process..
Process 11422 stopped
$Stopping wsdserverctl: 
Stopping WSD Server process..
Process 11425 stopped


In [9]:
df=pd.DataFrame(cptnm,cpt_sem)
df.reset_index(level=0, inplace=True)
df.rename(columns={0:'Concepts','index': 'semantic_type'}, inplace=True)
df=df.drop_duplicates()
df

,semantic_type,Concepts
0,Gene or Genome,brca 1 gene
1,Functional Concept,causes
3,Conceptual Entity,cause
4,Finding,breast cancer predisposition
5,Neoplastic Process,ovarian cancer
